In [47]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping


data_path = "carla/fixeddelta020take2.log"

def hex_to_bin(hex_num):
    
    binary_value = bin(int(str(hex_num), 16))[2:]
    
    return binary_value

def int_to_bin(int_num):
    
    binary_value = bin(int_num)[2:]
    
    return binary_value

hex_to_dec = lambda x: int(x, 16)
dec_to_hex = lambda x : hex(int(x))[2:]

In [48]:
columns = ['Timestamp', 'ID', 'DLC', 'Payload']

# Read the file into a list of lines
with open(data_path, 'r') as file:
    lines = file.readlines()

# Parse each line and extract the relevant information
data = []
for line in lines:
    parts = line.strip().split()
    timestamp = float(parts[0][1:-1])  # Remove parentheses
    bus_name = parts[1]
    ID = parts[2]
    DLC = int(parts[3][1:-1])  # Remove brackets and convert to integer
    payload = ''.join(parts[4:])  # Concatenate payload
    data.append([timestamp, ID, DLC, payload])

# Create a DataFrame from the parsed data
df = pd.DataFrame(data, columns=columns)

In [49]:
df['ID'] = df['ID'].apply(hex_to_dec)
df['Payload'] = df['Payload'].apply(hex_to_dec)
df = df.assign(IAT=df['Timestamp'].diff().fillna(0))

In [50]:
df.drop(columns=['Timestamp'], inplace=True)

In [51]:
df.shape

(115095, 4)

In [52]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [35]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [42]:
input_dim = X_train.shape[1]

In [43]:
model = Sequential()

##Encoder
model.add(Dense(input_dim, input_shape=(input_dim, ), activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(2, activation='relu'))

##Bottleneck
model.add(Dense(1, activation='relu'))

##Decoder
model.add(Dense(2, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(input_dim))


In [46]:
EPOCHS = 1000
BATCH_SIZE = 32
LOSS = 'mse'

def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch % 20 == 0:
        return lr / 10
    return lr

lr_callback = LearningRateScheduler(lr_scheduler, verbose=1)

# Define early stopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=60, restore_best_weights=True)

model.compile(optimizer='adam', loss=LOSS)

history = model.fit(X_train, X_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, X_test), callbacks=[lr_callback, early_stopper])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/1000


2024-03-01 00:28:27.817193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2878/2878 [==============================] - ETA: 0s - loss: 0.7501

2024-03-01 00:28:52.301196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2878/2878 [==============================] - 27s 8ms/step - loss: 0.7501 - val_loss: 0.7489 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/1000
2878/2878 [==============================] - 23s 8ms/step - loss: 0.7501 - val_loss: 0.7488 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/1000
2878/2878 [==============================] - 22s 8ms/step - loss: 0.7501 - val_loss: 0.7489 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/1000
2878/2878 [==============================] - 24s 8ms/step - loss: 0.7501 - val_loss: 0.7488 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 5/1000
2878/2878 [==============================] - 23s 8ms/step - loss: 0.7501 - val_loss: 0.7489 - lr: 0.0010

Epoch 6: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 6/1000
2878/2878 

KeyboardInterrupt: 

In [ ]:
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.savefig('ae_loss_chart.png')
# model.save('ae.h5')